In [1]:
import pandas as pd
import numpy as np
import plotly.express as px
from bs4 import BeautifulSoup
import requests
import os
import pdfplumber
from pdf2image import convert_from_path
import pytesseract
import time
import re

import warnings
warnings.filterwarnings("ignore")

In [2]:
df = pd.read_csv("FastTrack_Updated_09-16-2024.csv")
df.head()

,Company,Landed Month / Year,Capital Investment,Incentivized Jobs,New Jobs,Project Type,County,County Tier,FJTAP,FIDP,ED,Grants Total
0,Ultium Cells LLC,4/16/2021,"$2,367,000,000","1,700","1,300",Recruitment,Maury,1,NaN,NaN,"$78,000,000","$78,000,000"
1,"Oracle America, Inc.",8/31/2021,"$1,350,000,000","5,989","8,500",Recruitment,Davidson,1,NaN,NaN,"$65,000,000","$65,000,000"
2,LG Chem America Inc.,11/21/2022,"$3,205,715,854",860,860,Recruitment,Montgomery,1,NaN,NaN,"$40,000,000","$40,000,000"
3,ALSAC St Jude Children's,11/30/2016,"$1,000,000,000","1,800",1800,Expansion,Shelby,2,NaN,NaN,"$36,000,000","$36,000,000"
4,General Motors LLC,10/20/2020,"$1,717,000,000","2,000",0,Expansion,Maury,1,"$35,000,000",NaN,NaN,"$35,000,000"


In [3]:
df.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 778 entries, 0 to 777
Data columns (total 12 columns):
 #   Column               Non-Null Count  Dtype 
---  ------               --------------  ----- 
 0   Company              778 non-null    object
 1   Landed Month / Year  778 non-null    object
 2   Capital Investment   778 non-null    object
 3   Incentivized Jobs    778 non-null    object
 4   New Jobs             778 non-null    object
 5   Project Type         778 non-null    object
 6   County               778 non-null    object
 7   County Tier          778 non-null    int64 
 8   FJTAP                522 non-null    object
 9   FIDP                 34 non-null     object
 10  ED                   272 non-null    object
 11  Grants Total         778 non-null    object
dtypes: int64(1), object(11)
memory usage: 73.1+ KB


In [4]:
# Lists for each region
region_1 = ["Anderson", "Blount", "Campbell", "Carter", "Claiborne", "Cocke", "Grainger", "Greene", "Hamblen", "Hancock", "Hawkins", "Jefferson", "Johnson", "Knox", "Loudon", "Monroe", "Morgan", "Roane", "Scott", "Sevier", "Sullivan", "Unicoi", "Union", "Washington"]
region_2 = ["Bledsoe", "Bradley", "Cannon", "Clay", "Coffee", "Cumberland", "DeKalb", "Fentress", "Franklin", "Grundy", "Hamilton", "Jackson", "Marion", "McMinn", "Meigs", "Overton", "Pickett", "Polk", "Putnam", "Rhea", "Sequatchie", "Van Buren", "Warren", "White"]
region_3 = ["Bedford", "Cheatham", "Davidson", "Dickson", "Giles", "Hickman", "Houston", "Humphreys", "Lawrence", "Lewis", "Lincoln", "Macon", "Marshall", "Maury", "Montgomery", "Moore", "Perry", "Robertson", "Rutherford", "Smith", "Stewart", "Sumner", "Trousdale", "Washington", "Wayne", "Williamson"]
region_4 = ["Benton", "Carroll", "Chester", "Crockett", "Decatur", "Dyer", "Fayette", "Gibson", "Hardeman", "Hardin", "Haywood", "Henderson", "Henry", "Lake", "Lauderdale", "Madison", "McNairy", "Obion", "Shelby", "Tipton", "Weakley"]

# Combined list for all regions
all_regions = region_1 + region_2 + region_3 + region_4

In [5]:
healthcare_data = pd.read_csv(r"C:\Users\emm7a\Desktop\Coding Library\Archive\COE_URP\Healthcare Prediction Data.csv")
fips_df = healthcare_data[['FIPS', 'County']]
fips_df = fips_df.drop_duplicates(subset=['FIPS'])

In [6]:
county_df = pd.merge(df, fips_df, on='County', how='left')
county_df.head()

,Company,Landed Month / Year,Capital Investment,Incentivized Jobs,New Jobs,Project Type,County,County Tier,FJTAP,FIDP,ED,Grants Total,FIPS
0,Ultium Cells LLC,4/16/2021,"$2,367,000,000","1,700","1,300",Recruitment,Maury,1,NaN,NaN,"$78,000,000","$78,000,000",47119.0
1,"Oracle America, Inc.",8/31/2021,"$1,350,000,000","5,989","8,500",Recruitment,Davidson,1,NaN,NaN,"$65,000,000","$65,000,000",47037.0
2,LG Chem America Inc.,11/21/2022,"$3,205,715,854",860,860,Recruitment,Montgomery,1,NaN,NaN,"$40,000,000","$40,000,000",47125.0
3,ALSAC St Jude Children's,11/30/2016,"$1,000,000,000","1,800",1800,Expansion,Shelby,2,NaN,NaN,"$36,000,000","$36,000,000",47157.0
4,General Motors LLC,10/20/2020,"$1,717,000,000","2,000",0,Expansion,Maury,1,"$35,000,000",NaN,NaN,"$35,000,000",47119.0


In [7]:
# Data Cleaning
county_df['Landed Month / Year'] = pd.to_datetime(county_df['Landed Month / Year'], format='%m/%d/%Y')
county_df['Capital Investment'] = county_df['Capital Investment'].replace({'\$': '', ',': ''}, regex=True).astype('int64')
county_df['Grants Total'] = county_df['Grants Total'].replace({'\$': '', ',': ''}, regex=True).astype('int64')
county_df['FJTAP'] = county_df['FJTAP'].fillna(0).replace({'\$': '', ',': ''}, regex=True).astype('int64')
county_df['FIDP'] = county_df['FIDP'].fillna(0).replace({'\$': '', ',': ''}, regex=True).astype('int64')
county_df['ED'] = county_df['ED'].fillna(0).replace({'\$': '', ',': ''}, regex=True).astype('int64')
county_df['Incentivized Jobs'] = county_df['Incentivized Jobs'].replace({'\$': '', ',': ''}, regex=True).astype('int64')
county_df['New Jobs'] = county_df['New Jobs'].replace({'\$': '', ',': ''}, regex=True).astype('int64')

In [8]:
regions = pd.read_excel("Forecasting_Employment.xlsx", sheet_name='Regions')
# Step 1: Melt the 'regions' DataFrame
melted_regions = regions.melt(var_name='Region', value_name='County')

# Step 2: Drop duplicates in case a county appears in more than one region
melted_regions = melted_regions.drop_duplicates(subset='County')

# Step 3: Map the Region to the County in 'county_df'
county_df['Region'] = county_df['County'].map(melted_regions.set_index('County')['Region'])

# Now 'county_df' will have a 'Region' column that corresponds to each 'County'

In [46]:
county_df.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 778 entries, 0 to 777
Data columns (total 15 columns):
 #   Column               Non-Null Count  Dtype         
---  ------               --------------  -----         
 0   Company              778 non-null    object        
 1   Landed Month / Year  778 non-null    datetime64[ns]
 2   Capital Investment   778 non-null    int64         
 3   Incentivized Jobs    778 non-null    int64         
 4   New Jobs             778 non-null    int64         
 5   Project Type         778 non-null    object        
 6   County               778 non-null    object        
 7   County Tier          778 non-null    int64         
 8   FJTAP                778 non-null    int64         
 9   FIDP                 778 non-null    int64         
 10  ED                   778 non-null    int64         
 11  Grants Total         778 non-null    int64         
 12  FIPS                 720 non-null    float64       
 13  Region               755 non-null  

In [10]:
def generate_county_map(df, values_column, values_label, title, color_scale):
    df1 = df.copy()
    # df1['FIPS'] = df1['FIPS'].apply(lambda x: int(x[:-2]))
    fig = px.choropleth(df1, 
                        geojson='https://raw.githubusercontent.com/plotly/datasets/master/geojson-counties-fips.json', 
                        locations='FIPS', 
                        color=values_column,
                        color_continuous_scale=color_scale,
                        range_color=(df1[values_column].min(), df1[values_column].max()),
                        scope="usa",
                        labels={values_column: values_label},
                        hover_data={'County': True}
                       )

    fig.update_geos(center=dict(lon=-86.25, lat=36), 
                    projection_scale=7)  # Adjust projection_scale as needed for zoom level
    fig.update_layout(
        title_text=title,
        title_x=0.5,  # Center the title horizontally
        title_y=0.9,
        title_font=dict(size=20),  # Increase the font size
        template="plotly_dark",
        height=600,
        width=800,
        margin=dict(l=0, r=0, t=0, b=0),  # Adjusted top margin to accommodate the larger title
        coloraxis_colorbar=dict(
            len=0.75,  # Decrease the length of the color scale to 50% of the plot area height
            orientation='v'  # Ensure it's vertical; change to 'h' for horizontal
        )
    )
    fig.show()

In [11]:
# Convert FIPS to integers (after handling missing FIPS)
county_df_cleaned = county_df.dropna(subset=['FIPS'])  # Drop rows with missing FIPS values
county_df_cleaned['FIPS'] = county_df_cleaned['FIPS'].astype(int)

# Generate the map
generate_county_map(county_df_cleaned, 'New Jobs', 'New Jobs', 'County Map - New Jobs Distribution', 'Blues')

In [12]:
county_df.head()

,Company,Landed Month / Year,Capital Investment,Incentivized Jobs,New Jobs,Project Type,County,County Tier,FJTAP,FIDP,ED,Grants Total,FIPS,Region
0,Ultium Cells LLC,2021-04-16,2367000000,1700,1300,Recruitment,Maury,1,0,0,78000000,78000000,47119.0,Region 3
1,"Oracle America, Inc.",2021-08-31,1350000000,5989,8500,Recruitment,Davidson,1,0,0,65000000,65000000,47037.0,Region 3
2,LG Chem America Inc.,2022-11-21,3205715854,860,860,Recruitment,Montgomery,1,0,0,40000000,40000000,47125.0,Region 3
3,ALSAC St Jude Children's,2016-11-30,1000000000,1800,1800,Expansion,Shelby,2,0,0,36000000,36000000,47157.0,Region 4
4,General Motors LLC,2020-10-20,1717000000,2000,0,Expansion,Maury,1,35000000,0,0,35000000,47119.0,Region 3


In [45]:
county_df.to_csv('FastTrack_Data_10-16-24.csv')

## Dashboard

In [43]:
import dash
from dash import dcc, html
import dash_bootstrap_components as dbc
import pandas as pd
import webbrowser
from threading import Timer
from dash.dependencies import Input, Output
import plotly.express as px
import plotly.graph_objects as go

# Function to open the browser automatically
def open_browser():
    webbrowser.open_new("http://127.0.0.1:8050/")

# Initialize the Dash app
app = dash.Dash(__name__, external_stylesheets=[dbc.themes.BOOTSTRAP])

# Assuming county_df is already loaded in your environment
# Create the 'Grant Type' column in the DataFrame
def create_grant_type_column(df):
    df['Grant Type'] = df.apply(lambda row: [grant for grant in ['FJTAP', 'FIDP', 'ED'] if row[grant] > 0], axis=1)
    return df

# Update county_df with the new Grant Type column
county_df = create_grant_type_column(county_df)

# Extracting min and max year for the year slider
min_year = county_df['Landed Month / Year'].dt.year.min()
max_year = county_df['Landed Month / Year'].dt.year.max()

# Summarize county_df by County, include the Region
def summarize_by_county(df):
    county_summary = df.groupby(['County', 'Region']).agg({
        'New Jobs': 'sum',
        'Capital Investment': 'sum',
        'FIPS': 'first'
    }).reset_index()

    # Format New Jobs and Capital Investment with comma and dollar sign
    county_summary['Total New Jobs'] = county_summary['New Jobs'].apply(lambda x: f"{x:,}")
    county_summary['Total Capital Investment'] = county_summary['Capital Investment'].apply(lambda x: f"${x:,.0f}")
    county_summary.sort_values(by='New Jobs', ascending=False, inplace=True)
    
    return county_summary

# Filter the dataframe based on selected filters
def filter_data(df, regions, project_types, year_range, grant_types):
    filtered_df = df.copy()

    # Filter by Region
    if regions:
        filtered_df = filtered_df[filtered_df['Region'].isin(regions)]

    # Filter by Project Type
    if project_types:
        filtered_df = filtered_df[filtered_df['Project Type'].isin(project_types)]

    # Filter by Year Range
    filtered_df = filtered_df[filtered_df['Landed Month / Year'].dt.year.between(year_range[0], year_range[1])]

    # Filter by Grant Type
    if grant_types:
        filtered_df = filtered_df[filtered_df['Grant Type'].apply(lambda grants: any(grant in grants for grant in grant_types))]

    return filtered_df

# Generate county map
def generate_county_map(df, values_column, title):
    fig = px.choropleth(df, 
                        geojson='https://raw.githubusercontent.com/plotly/datasets/master/geojson-counties-fips.json', 
                        locations='FIPS', 
                        color=values_column,
                        color_continuous_scale='Blues',
                        scope="usa",
                        labels={values_column: values_column},
                        hover_data={'County': True})

    fig.update_geos(center=dict(lon=-85.90, lat=35.5), projection_scale=7.25)
    fig.update_layout(
        autosize=True,
        title_text=title,
        title_x=0.5,
        title_y=0.95,
        title_font=dict(size=24),
        template="plotly_dark",
        height=600,
        margin=dict(l=0, r=0, t=0, b=0),
        coloraxis_colorbar=dict(
            len=0.8,
            x=0.5,
            y=0.05,
            orientation='h',
            title_side="top"
        )
    )
    
    return fig

# Generate scatter plot for New Jobs and Capital Investment
def generate_scatter_plot(df, y_column, title):
    df_yearly = df.groupby(df['Landed Month / Year'].dt.year)[y_column].sum().reset_index()

    fig = go.Figure()
    fig.add_trace(go.Scatter(
        x=df_yearly['Landed Month / Year'],
        y=df_yearly[y_column],
        mode='lines+markers',
        name=title,
        line=dict(shape='linear')
    ))

    fig.update_layout(
        template="plotly_dark",
        title=title,
        xaxis_title="Year",
        yaxis_title=y_column,
        height=400,
        margin=dict(l=10, r=10, t=50, b=30)
    )
    
    return fig

# Layout for the app
app.layout = dbc.Container(
    fluid=True,
    style={"backgroundColor": "#9e9ead"},
    children=[
        # Title row
        dbc.Row(
            dbc.Col(
                html.H1("Tennessee: Forecasting Employment Growth", className="text-center"),
                width={"size": 6, "offset": 3},
                className="d-flex justify-content-center align-items-center mb-4"
            ),
        ),
        
        # Filters panel row
        dbc.Row(
            dbc.Col(
                dbc.Card(
                    dbc.CardBody(
                        dbc.Row(
                            [
                                dbc.Col(dcc.Dropdown(id='region-filter', options=[{'label': f'Region {i}', 'value': f'Region {i}'} for i in range(1, 5)], multi=True, placeholder='Filter Region'), width=3),
                                dbc.Col(dcc.Dropdown(id='project-type-filter', options=[{'label': ptype, 'value': ptype} for ptype in ['Recruitment', 'Expansion', 'Expansion New Location', 'New Startup']], multi=True, placeholder='Filter Project Type'), width=3),
                                dbc.Col(dcc.RangeSlider(id='year-slider', min=min_year, max=max_year, marks={str(year): str(year) for year in range(min_year, max_year + 1)}, value=[min_year, max_year], step=1), width=4),
                                dbc.Col(dcc.Dropdown(id='grant-type-filter', options=[{'label': grant, 'value': grant} for grant in ['FJTAP', 'FIDP', 'ED']], multi=True, placeholder='Filter Grant Type'), width=2)
                            ]
                        )
                    ),
                    className="mb-4"
                ),
                width=12
            )
        ),

        # Map and top counties table row
        dbc.Row(
            [
                dbc.Col(dcc.Graph(id='county-map'), width=8, style={"width": "65vw", "margin-left": "0px", "margin-right": "auto"}),
                dbc.Col(
                    dbc.Container(
                        [
                            html.Div([
                                html.H4("Top Counties for New Jobs"),
                                dbc.Table.from_dataframe(pd.DataFrame(), bordered=True, hover=True, responsive=True, id='top-counties-table')
                            ], style={"margin-bottom": "20px"})
                        ]
                    ),
                    width=4
                )
            ]
        ),

        # Scatter plots row and statistics box
        dbc.Row(
            [
                dbc.Col(dcc.Graph(id='new-jobs-plot'), width=4, style={"margin-top": "30px"}),
                dbc.Col(dcc.Graph(id='capital-investment-plot'), width=4, style={"margin-top": "30px"}),
                dbc.Col(
                    html.Div(
                        [html.H4("Statistics"), html.Div(id='statistics-box')],
                        style={"border": "1px solid #ccc", "padding": "10px", "background-color": "#f8f9fa", "margin-top": "-22vh"}
                    ),
                    width=4
                )
            ]
        )
    ]
)

# Updated callback function to create compartmentalized statistics box content
@app.callback(
    [Output('county-map', 'figure'),
     Output('top-counties-table', 'children'),
     Output('new-jobs-plot', 'figure'),
     Output('capital-investment-plot', 'figure'),
     Output('statistics-box', 'children')],
    [Input('region-filter', 'value'),
     Input('project-type-filter', 'value'),
     Input('year-slider', 'value'),
     Input('grant-type-filter', 'value')]
)
def update_dashboard(selected_regions, selected_project_types, selected_years, selected_grants):
    filtered_df = filter_data(county_df, selected_regions, selected_project_types, selected_years, selected_grants)
    county_summary_df = summarize_by_county(filtered_df)

    # Generate updated visuals
    county_map_fig = generate_county_map(county_summary_df, 'New Jobs', 'County-wise New Jobs')
    top_counties_table = dbc.Table.from_dataframe(
        county_summary_df[['County', 'Region', 'Total New Jobs', 'Total Capital Investment']].head(5),
        bordered=True,
        hover=True,
        responsive=True
    )
    new_jobs_plot_fig = generate_scatter_plot(filtered_df, 'New Jobs', 'New Jobs Over Time')
    investment_plot_fig = generate_scatter_plot(filtered_df, 'Capital Investment', 'Capital Investment Over Time')

    # Statistics content calculation
    unique_counties = filtered_df['FIPS'].nunique()
    total_projects = len(filtered_df)

    # Grant type counts - ensure all grant types are represented even if they have 0 count
    grant_type_counts = {grant: (filtered_df[grant] > 0).sum() if grant in filtered_df else 0 for grant in ['FJTAP', 'FIDP', 'ED']}
    
    # Ensure regions are always displayed in order 1 through 4
    region_order = [f'Region {i}' for i in range(1, 5)]
    region_counts = {region: filtered_df['Region'].value_counts().get(region, 0) for region in region_order}

    # Ensure all project types are represented even if they have 0 count
    all_project_types = ['Recruitment', 'Expansion', 'Expansion New Location', 'New Startup']
    project_type_counts = {ptype: filtered_df['Project Type'].value_counts().get(ptype, 0) for ptype in all_project_types}

    # Compartmentalized statistics content using Cards
    statistics_content = dbc.Container(
        [
            dbc.Card(
                dbc.CardBody([
                    html.H5(f"Unique Counties Analyzed: {unique_counties}", className="card-title"),
                    html.H5(f"Total Number of Projects: {total_projects}", className="card-title")
                ]),
                className="mb-3"
            ),
            dbc.Card(
                dbc.CardBody([
                    html.H6("Projects Breakdown by Grant Type:", className="card-subtitle"),
                    html.Ul([html.Li(f"{grant}: {count}") for grant, count in grant_type_counts.items()])
                ]),
                className="mb-3"
            ),
            dbc.Card(
                dbc.CardBody([
                    html.H6("Projects Breakdown by Region:", className="card-subtitle"),
                    html.Ul([html.Li(f"{region}: {region_counts[region]}") for region in region_order])
                ]),
                className="mb-3"
            ),
            dbc.Card(
                dbc.CardBody([
                    html.H6("Projects Breakdown by Project Type:", className="card-subtitle"),
                    html.Ul([html.Li(f"{ptype}: {count}") for ptype, count in project_type_counts.items()])
                ]),
                className="mb-3"
            )
        ],
        style={"padding": "10px"}
    )

    return county_map_fig, top_counties_table, new_jobs_plot_fig, investment_plot_fig, statistics_content

# Run the app
if __name__ == "__main__":
    Timer(1, open_browser).start()
    app.run_server(debug=True)

In [14]:
county_df.columns

Index(['Company', 'Landed Month / Year', 'Capital Investment',
       'Incentivized Jobs', 'New Jobs', 'Project Type', 'County',
       'County Tier', 'FJTAP', 'FIDP', 'ED', 'Grants Total', 'FIPS', 'Region',
       'Grant Type'],
      dtype='object')

## TN Investco Data

In [15]:
# URL of the page to scrape
url = "https://www.tn.gov/transparenttn/state-financial-overview/open-ecd/openecd/openecd-tninvestco.html"

# Request the content of the webpage
response = requests.get(url)
response.raise_for_status()  # Check for errors in request

# Parse the HTML content
soup = BeautifulSoup(response.text, 'html.parser')

# Find all <a> elements inside <li> tags that contain "TNInvestco Annual Report"
reports = soup.find_all('a', href=True, text=lambda x: x and 'TNInvestco Annual Report' in x)

# Folder to store downloaded PDFs
os.makedirs('tninvestco_reports', exist_ok=True)

# List to store the extracted contents of each PDF
pdf_contents = []

# Function to extract text from PDF (with OCR fallback)
def extract_text_from_pdf(pdf_path):
    try:
        with pdfplumber.open(pdf_path) as pdf:
            all_text = ''
            for page_num, page in enumerate(pdf.pages):
                text = page.extract_text()
                if text:
                    all_text += text
                else:
                    # If no text is found, convert the page to an image and apply OCR
                    print(f"Page {page_num} might be image-based. Applying OCR...")
                    images = convert_from_path(pdf_path, first_page=page_num + 1, last_page=page_num + 1)
                    for image in images:
                        ocr_text = pytesseract.image_to_string(image)
                        all_text += ocr_text
            return all_text
    except Exception as e:
        print(f"Error processing PDF {pdf_path}: {e}")
        return None

# Loop through the report links from 2022 to 2014
for report in reports:
    report_text = report.get_text()
    
    # Extract the year from the report text
    if '2022' in report_text or '2021' in report_text or '2020' in report_text or '2019' in report_text \
            or '2018' in report_text or '2017' in report_text or '2016' in report_text \
            or '2015' in report_text or '2014' in report_text:
        
        # Handle special cases for 2015 and 2014
        year = report_text.split()[-1]
        if '2015' in report_text or '2014' in report_text:
            # Special URL for 2015 and 2014
            report_url = report['href']
            if report_url.startswith('/'):
                report_url = "https://www.tnecdit.net" + report_url
        else:
            # Normal URL structure for other years
            report_url = "https://www.tn.gov" + report['href']
        
        try:
            # Download the PDF with a delay between requests
            print(f"Downloading {report_url}...")
            pdf_response = requests.get(report_url, timeout=10)  # Timeout of 10 seconds for each request
            pdf_response.raise_for_status()  # Check for errors
            
            # Save the PDF file
            pdf_path = f'tninvestco_reports/TNInvestco_Annual_Report_{year}.pdf'
            with open(pdf_path, 'wb') as file:
                file.write(pdf_response.content)
            
            print(f'Downloaded {pdf_path}')
            
            # Extract text from the downloaded PDF
            extracted_text = extract_text_from_pdf(pdf_path)
            if extracted_text:
                pdf_contents.append({
                    'year': year,
                    'content': extracted_text
                })
            
            print(f'Extracted content from {pdf_path}')

        except requests.exceptions.RequestException as e:
            print(f"Error downloading {report_url}: {e}")

        # Introduce a delay (e.g., 3 seconds) to avoid overwhelming the server
        time.sleep(3)

# Now pdf_contents contains the extracted text from each PDF

Downloaded tninvestco_reports/TNInvestco_Annual_Report_2022.pdf
Extracted content from tninvestco_reports/TNInvestco_Annual_Report_2022.pdf
Downloaded tninvestco_reports/TNInvestco_Annual_Report_2021.pdf
Extracted content from tninvestco_reports/TNInvestco_Annual_Report_2021.pdf
Downloaded tninvestco_reports/TNInvestco_Annual_Report_2020.pdf
Extracted content from tninvestco_reports/TNInvestco_Annual_Report_2020.pdf
Downloaded tninvestco_reports/TNInvestco_Annual_Report_2019.pdf
Extracted content from tninvestco_reports/TNInvestco_Annual_Report_2019.pdf
Downloaded tninvestco_reports/TNInvestco_Annual_Report_2018.pdf
Extracted content from tninvestco_reports/TNInvestco_Annual_Report_2018.pdf
Downloaded tninvestco_reports/TNInvestco_Annual_Report_2017.pdf
Extracted content from tninvestco_reports/TNInvestco_Annual_Report_2017.pdf
Downloaded tninvestco_reports/TNInvestco_Annual_Report_2016.pdf
Extracted content from tninvestco_reports/TNInvestco_Annual_Report_2016.pdf
Downloaded tninvestc

---------------------------------------------------------------------------
TypeError                                 Traceback (most recent call last)
Cell In[13], line 176, in update_dashboard(
    selected_regions=None,
    selected_project_types=None,
    selected_years=[2016, 2024],
    selected_grants=None
)
    173 county_summary_df = summarize_by_county(filtered_df)
    175 # Generate updated visuals
--> 176 county_map_fig = generate_county_map(county_summary_df, 'New Jobs', 'County-wise New Jobs')
        county_summary_df =         County    Region  New Jobs  Capital Investment     FIPS  \
17    Davidson  Region 3     22905          2950191918  47037.0   
73      Shelby  Region 4     13679          2864416763  47157.0   
54       Maury  Region 3      5882          5722406319  47119.0   
30    Hamilton  Region 2      5572          1056809657  47065.0   
85  Williamson  Region 3      4623           448090595  47187.0   
..         ...       ...       ...                 ...    

In [16]:
# Keywords for classification of business (multi-word first)
classification_keywords = [
    "Healthcare Technology", "Security Staffing","Information Technology", "Computer Design", "Healthcare IT/Communications",
    "Supply Chain","Education Technology", "Medical Device", "Medial Device", "Consumer Technology", "Mobile Technology", 
    "Life Sciences/Biotech", "Medical Technology", "Automotive Technology", "Industrial Technology", "Life Sciences", 
    "Transformational Technology", "Applied Technology", "Business Technology",
    "Social Media", "Healthcare Services", "Healthcare Communications", "Information Services", "Technology Development", 
    "Health Informatics", "Renewable Energy", "Clean Energy", "Healthcare IT", "Pharmaceuticals", "Business Services", 
    "Biopharma", "Bioscience", "Logistics", "Electronics", "Manufacturing", "Chemical", "Healthcare", 
    "Software", "Transportation", "Automotive", "Informatics", "Diagnostics"
]

# Compile the pattern for classification words (multi-word first, case-insensitive)
classification_pattern = r"\b(" + "|".join(re.escape(keyword) for keyword in classification_keywords) + r")\b"

# Data container for parsed rows
parsed_data = []

# Function to extract location and remaining details from the row
def extract_location_and_rest(line, classification_match):
    classification_idx = classification_match.end()
    
    # Everything after the classification match
    remaining_line = line[classification_idx:].strip()
    
    # Split by spaces, handling cases where some locations might be two words
    parts = remaining_line.split()
    
    # Find the location (it stops when we encounter the investment, which starts with $)
    location = []
    i = 0
    while i < len(parts) and not parts[i].startswith('$'):
        location.append(parts[i])
        i += 1

    location = " ".join(location)
    
    # Remaining parts (from the investment value onwards)
    remaining_parts = parts[i:]
    return location, remaining_parts

# Function to parse the extracted PDF content
def parse_pdf_content(content, year):
    lines = content.strip().split("\n")
    
    current_fund = None  # To keep track of the current fund
    
    for line_num, line in enumerate(lines):
        # Normalize multiple spaces to a single space
        line = re.sub(r'\s+', ' ', line).strip()

        # Skip non-business-related lines
        if "Cumulative Amount" in line or "TNInvestco ANNUAL REPORT" in line:
            continue

        # Look for Fund markers (line before '> 15% of')
        if "> 15% of" in line:
            # The line before the current line is the Fund
            current_fund = lines[line_num - 1].strip()
            continue
        
        # Search for the classification keyword in the line
        classification_match = re.search(classification_pattern, line, re.IGNORECASE)
        
        if classification_match:
            classification = classification_match.group(1)
            
            # Everything before the classification is the qualified business name
            qualified_business = line[:classification_match.start()].strip()
            
            # Extract the location and remaining parts of the line
            location, remaining = extract_location_and_rest(line, classification_match)

            # Ensure we have enough data in the remaining parts
            if len(remaining) >= 5:
                total_investment = remaining[0].replace("$", "").replace(",", "")
                retained_jobs = remaining[1]
                total_jobs = remaining[2]
                women_minority_employees = remaining[3]
                follow_on_capital = remaining[4].replace("$", "").replace(",", "")

                # Append the parsed row to the parsed_data list
                parsed_data.append([
                    qualified_business, classification, location, total_investment,
                    retained_jobs, total_jobs, women_minority_employees, follow_on_capital,
                    year, current_fund
                ])
            else:
                print(f"Error: Insufficient data found in line: {line}")
        else:
            print(f"Classification not found in line: {line}")

# Process each PDF content in pdf_contents
for pdf in pdf_contents:
    print(f"Processing year {pdf['year']}...")
    parse_pdf_content(pdf['content'], pdf['year'])

# Convert parsed data into a pandas DataFrame
columns = [
    "Qualified Business", "Classification of Business", "Location",
    "Total Investment (since inception)", "Retained Jobs", "Total Jobs",
    "Women and Minority Employees", "Private Follow-on Capital Raised by Invested Companies",
    "Year", "Program"
]

df_new = pd.DataFrame(parsed_data, columns=columns)

Processing year 2022...
Classification not found in line: *The information shown below reflects figures reported by each TNInvestco related to investment activity. The summary section reflects adjusted figures of the TNInvestco program where the duplication of jobs and follow-on capital have been removed.
Classification not found in line: TNInvestco
Classification not found in line: Council & Enhanced
Classification not found in line: Women and Private Follow-on Capital Amount of Capital Invested
Classification not found in line: Total Investment Minority Raised by Invested Invested in each Capital in Performance Regarding
Classification not found in line: Qualified Business Classification of Business Location (since inception) Retained Jobs Total Jobs Employees Companies TNInvestco Single Co. Compliance
Classification not found in line: TNInvestco
Classification not found in line: Innova
Classification not found in line: Women and Private Follow-on Capital Amount of Capital Invested
C

In [17]:
df_new.head()

,Qualified Business,Classification of Business,Location,Total Investment (since inception),Retained Jobs,Total Jobs,Women and Minority Employees,Private Follow-on Capital Raised by Invested Companies,Year,Program
0,ABT Molecular,Healthcare,Louisville,525000,30,27,4,29741173,2022,Council & Enhanced
1,Advanced Network Solutions,Computer Design,Nashville,1560933,29,44,16,10046795,2022,Council & Enhanced
2,Amplion (f/k/a Dalcon),Information Technology,Nashville,800001,18,11,6,15143130,2022,Council & Enhanced
3,Energy Source Partners,Renewable Energy,Nashville,1000000,3,1,0,0,2022,Council & Enhanced
4,Fsi,Supply Chain,Nashville,2100000,22,52,29,6125109,2022,Council & Enhanced


In [18]:
df_unique = df_new.drop_duplicates(subset=[
    "Qualified Business", "Classification of Business", "Location",
    "Total Investment (since inception)", "Retained Jobs", "Total Jobs",
    "Women and Minority Employees", "Private Follow-on Capital Raised by Invested Companies", "Program"
]).reset_index(drop=True)

In [19]:
df_unique.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 963 entries, 0 to 962
Data columns (total 10 columns):
 #   Column                                                  Non-Null Count  Dtype 
---  ------                                                  --------------  ----- 
 0   Qualified Business                                      963 non-null    object
 1   Classification of Business                              963 non-null    object
 2   Location                                                963 non-null    object
 3   Total Investment (since inception)                      963 non-null    object
 4   Retained Jobs                                           963 non-null    object
 5   Total Jobs                                              963 non-null    object
 6   Women and Minority Employees                            963 non-null    object
 7   Private Follow-on Capital Raised by Invested Companies  963 non-null    object
 8   Year                                              

In [20]:
df_unique.sort_values(by=["Qualified Business", 'Year'])

,Qualified Business,Classification of Business,Location,Total Investment (since inception),Retained Jobs,Total Jobs,Women and Minority Employees,Private Follow-on Capital Raised by Invested Companies,Year,Program
807,ABT Molecular,Healthcare,Louisville,525000,30,39,9,22005173,2014,Council & Enhanced
832,ABT Molecular,Medical Device,Louisville,525000,30,39,9,22005173,2014,Limestone
588,ABT Molecular,Healthcare,Louisville,525000,30,30,6,22755173,2015,Remaining
627,ABT Molecular,Medical Device,Louisville,525000,30,30,6,22755173,2015,Remaining
482,ABT Molecular,Healthcare,Louisville,525000,29,29,5,24691173,2016,Council & Enhanced
...,...,...,...,...,...,...,...,...,...,...
301,iScreen Vision,Medical Device,Cordova,982380,5,11,8,12543748,2020,Innova
32,iScreen Vision,Medical Device,Cordova,1673648,8.5,11,7,10244253,2022,Innova
83,iScreen Vision,Medical Device,Cordova,982380,5,11,8,12543748,2022,MB Ventures
786,onFocus,Healthcare IT,Brentwood,392809,17,19,5,4548562,2015,Remaining


In [21]:
# Ensure the relevant columns are numeric for calculations
df_unique['Year'] = pd.to_numeric(df_unique['Year'], errors='coerce')
df_unique['Total Investment (since inception)'] = pd.to_numeric(df_unique['Total Investment (since inception)'], errors='coerce')
df_unique['Private Follow-on Capital Raised by Invested Companies'] = pd.to_numeric(df_unique['Private Follow-on Capital Raised by Invested Companies'], errors='coerce')
df_unique['Total Jobs'] = pd.to_numeric(df_unique['Total Jobs'], errors='coerce')

# Sort the dataframe by 'Qualified Business' and 'Year' to ensure correct order for calculation
df_unique = df_unique.sort_values(by=['Qualified Business', 'Year'])

# Calculate the year-over-year differences
df_unique['Investment Amount Added'] = df_unique.groupby('Qualified Business')['Total Investment (since inception)'].diff().fillna(0)
df_unique['Change in Total Jobs'] = df_unique.groupby('Qualified Business')['Total Jobs'].diff().fillna(0)
df_unique['Follow-on Capital Raised Added'] = df_unique.groupby('Qualified Business')['Private Follow-on Capital Raised by Invested Companies'].diff().fillna(0)

# For each business and year, replicate the calculations across duplicate rows in the same year
df_unique[['Investment Amount Added', 'Change in Total Jobs', 'Follow-on Capital Raised Added']] = df_unique.groupby(
    ['Qualified Business', 'Year']
)[['Investment Amount Added', 'Change in Total Jobs', 'Follow-on Capital Raised Added']].transform('first')

In [22]:
df_unique.head(10)

,Qualified Business,Classification of Business,Location,Total Investment (since inception),Retained Jobs,Total Jobs,Women and Minority Employees,Private Follow-on Capital Raised by Invested Companies,Year,Program,Investment Amount Added,Change in Total Jobs,Follow-on Capital Raised Added
807,ABT Molecular,Healthcare,Louisville,525000,30,39.0,9,22005173.0,2014,Council & Enhanced,0.0,0.0,0.0
832,ABT Molecular,Medical Device,Louisville,525000,30,39.0,9,22005173.0,2014,Limestone,0.0,0.0,0.0
588,ABT Molecular,Healthcare,Louisville,525000,30,30.0,6,22755173.0,2015,Remaining,0.0,-9.0,750000.0
627,ABT Molecular,Medical Device,Louisville,525000,30,30.0,6,22755173.0,2015,Remaining,0.0,-9.0,750000.0
482,ABT Molecular,Healthcare,Louisville,525000,29,29.0,5,24691173.0,2016,Council & Enhanced,0.0,-1.0,1936000.0
505,ABT Molecular,Medical Device,Louisville,525000,30,29.0,5,24691173.0,2016,Limestone,0.0,-1.0,1936000.0
263,ABT Molecular,Medical Device,Louisville,525000,30,27.0,4,29741173.0,2020,Innova,0.0,-2.0,5050000.0
0,ABT Molecular,Healthcare,Louisville,525000,30,27.0,4,29741173.0,2022,Council & Enhanced,0.0,0.0,0.0
45,ABT Molecular,Medical Device,Louisville,525000,30,27.0,4,29741173.0,2022,Limestone,0.0,0.0,0.0
693,Acacia Interactive,Information Services,Nashville,216710,0,0.0,0,30000.0,2015,Remaining,0.0,0.0,0.0


In [23]:
# Step 1: Identify the first year for each business
df_unique['is_first_year'] = df_unique.groupby('Qualified Business')['Year'].transform('min') == df_unique['Year']

# Step 2: Drop rows where all calculated columns are zero, except for the first year of the business
df_cleaned = df_unique[~(
    (df_unique['Investment Amount Added'] == 0) & 
    (df_unique['Change in Total Jobs'] == 0) & 
    (df_unique['Follow-on Capital Raised Added'] == 0) &
    ~df_unique['is_first_year']  # Keep the first year rows
)]

# Step 3: Drop the 'is_first_year' helper column
df_cleaned = df_cleaned.drop(columns=['is_first_year']).reset_index(drop=True)

In [24]:
df_cleaned.head(10)

,Qualified Business,Classification of Business,Location,Total Investment (since inception),Retained Jobs,Total Jobs,Women and Minority Employees,Private Follow-on Capital Raised by Invested Companies,Year,Program,Investment Amount Added,Change in Total Jobs,Follow-on Capital Raised Added
0,ABT Molecular,Healthcare,Louisville,525000,30,39.0,9,22005173.0,2014,Council & Enhanced,0.0,0.0,0.0
1,ABT Molecular,Medical Device,Louisville,525000,30,39.0,9,22005173.0,2014,Limestone,0.0,0.0,0.0
2,ABT Molecular,Healthcare,Louisville,525000,30,30.0,6,22755173.0,2015,Remaining,0.0,-9.0,750000.0
3,ABT Molecular,Medical Device,Louisville,525000,30,30.0,6,22755173.0,2015,Remaining,0.0,-9.0,750000.0
4,ABT Molecular,Healthcare,Louisville,525000,29,29.0,5,24691173.0,2016,Council & Enhanced,0.0,-1.0,1936000.0
5,ABT Molecular,Medical Device,Louisville,525000,30,29.0,5,24691173.0,2016,Limestone,0.0,-1.0,1936000.0
6,ABT Molecular,Medical Device,Louisville,525000,30,27.0,4,29741173.0,2020,Innova,0.0,-2.0,5050000.0
7,Acacia Interactive,Information Services,Nashville,216710,0,0.0,0,30000.0,2015,Remaining,0.0,0.0,0.0
8,Advanced Catheter Therapies,Medical Device,Chattanooga,450000,1,4.5,0,2886000.0,2014,Innova,0.0,0.0,0.0
9,Advanced Catheter Therapies,Medical Device,Chattanooga,450000,1,5.0,0,2886000.0,2015,Remaining,0.0,0.5,0.0


In [25]:
df_cleaned.columns

Index(['Qualified Business', 'Classification of Business', 'Location',
       'Total Investment (since inception)', 'Retained Jobs', 'Total Jobs',
       'Women and Minority Employees',
       'Private Follow-on Capital Raised by Invested Companies', 'Year',
       'Program', 'Investment Amount Added', 'Change in Total Jobs',
       'Follow-on Capital Raised Added'],
      dtype='object')

In [26]:
df_format = df_cleaned[['Qualified Business', 'Classification of Business', 'Location', 'Program', 
                        'Total Investment (since inception)', 'Investment Amount Added', 'Retained Jobs', 'Total Jobs', 
                        'Change in Total Jobs', 'Women and Minority Employees', 
                        'Private Follow-on Capital Raised by Invested Companies', 'Follow-on Capital Raised Added', 'Year'
                        ]]

In [27]:
df_format.to_excel("TNInvestco_data_Sep24.xlsx")